# Your Title Here

**Name(s)**: Landon Marchetti and Lyn Mansfield

**Website Link**: (your website link)

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from matplotlib.backends.backend_pdf import PdfPages

import plotly.express as px
pd.options.plotting.backend = 'plotly'

from dsc80_utils import * # Feel free to uncomment and use this.

In [2]:
%load_ext autoreload
%autoreload 2

## Step 1: Introduction

# TODO

Which legends are the most meta-defining? Difference in win rate between when it's banned vs when it's not

How important is the first kill of a game? 
- not important at all actually lol, the winrates are all the same no matter what

Which characters are the most banned? Which characters are the best?

What archetype of character's performance matters the most?

How important is early success?

## Step 2: Data Cleaning and Exploratory Data Analysis

# TODO
What are "partial" rows missing from their data? <== me!

In [113]:
# TODO

from pathlib import Path

dataset_path = Path("./data/2025_LoL_clean_stats.csv")
# /data/2025_LoL_clean_stats.csv
data = pd.read_csv(dataset_path)
data

,gameid,datacompleteness,url,league,...,monsterkills,monsterkillsownjungle,monsterkillsenemyjungle,cspm
0,LOLTMNT03_179647,complete,NaN,LFL2,...,0,-1,-1,8.82
1,LOLTMNT03_179647,complete,NaN,LFL2,...,132,-1,-1,5.39
2,LOLTMNT03_179647,complete,NaN,LFL2,...,0,-1,-1,7.88
...,...,...,...,...,...,...,...,...,...
60177,LOLTMNT06_120342,complete,NaN,CD,...,0,-1,-1,1.10
60178,LOLTMNT06_120342,complete,NaN,CD,...,159,-1,-1,34.30
60179,LOLTMNT06_120342,complete,NaN,CD,...,222,-1,-1,36.61


In [ ]:
type_frequencies = data.dtypes.value_counts()
type_frequencies.index = type_frequencies.index.map(str)
fig = px.bar(type_frequencies, 
	labels={'index': 'Data type', 'value': 'Frequency'}
	)
fig.update_layout(
    title={
        'text': 'Frequencies of Relevant Data Types',
        'font': {'size': 24}
    }
)
fig.show()

In [110]:
num_missing = data.apply(pd.isna).sum()
num_missing = num_missing / data.shape[0] * 100

fig = px.bar(num_missing[num_missing != 0].sort_values(), 
	labels={'index': 'Column name', 'value': '% missing'},
	title='Data missing from partial rows'
	)
fig.show()

In [7]:
team_rows = complete_samples.query("playername == ''")
team_rows

,gameid,datacompleteness,url,league,...,deathsat25,opp_killsat25,opp_assistsat25,opp_deathsat25
10,LOLTMNT03_179647,complete,,LFL2,...,10,10,26,3
11,LOLTMNT03_179647,complete,,LFL2,...,3,3,5,10
22,LOLTMNT06_96134,complete,,LFL2,...,8,7,12,15
...,...,...,...,...,...,...,...,...,...
167,LOLTMNT03_180652,complete,,LCKC,...,3,3,9,3
178,LOLTMNT03_180657,complete,,LCKC,...,7,7,7,5
179,LOLTMNT03_180657,complete,,LCKC,...,5,5,10,7


## Step 3: Assessment of Missingness

In [115]:
# TODO

def univariate_plots(df, title):
    pdf_pages = PdfPages(f'univariate_plots_{title}.pdf')

    plt.figure()

    for column in df.columns:
        fig, ax = plt.subplots()
        ax.hist(df[column])
        ax.set_title(f'Graph of {column}')
        pdf_pages.savefig(fig)
        plt.close(fig)
    
    pdf_pages.close()

def bivariate_plot(df, categorical_column, target_column, title):
    pdf_pages = PdfPage(f'bivariate_plots_{title}.pdf')

    plt.figure()
    
    for column in df.columns:
        fig, ax = plt.subplots()
        ax.hist(df[column])
        ax.set_title(f'Graph of {column}')
        pdf_pages.savefig(fig)
        plt.close(fig)
    
    pdf_pages.close()

In [118]:
univariate_plots(data[['dpm', 'damagetakenperminute', 'damagemitigatedperminute', 'earned gpm']], 'LoL_columns')

<Figure size 1000x500 with 0 Axes>

## Step 4: Hypothesis Testing

In [ ]:
# TODO

## Step 5: Framing a Prediction Problem

In [ ]:
# TODO
Use one-hot encoding!

## Step 6: Baseline Model

In [ ]:
# TODO

## Step 7: Final Model

In [ ]:
# TODO

## Step 8: Fairness Analysis

In [ ]:
# TODO